In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import sys
import os
import xgboost as xgb 
from importlib import reload
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error
from collections import namedtuple
import time
import pickle
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils import shuffle
codebase_path = '/data/home/wpw035/Codebase'
sys.path.insert(0, codebase_path) #add path to my codebase models

In [2]:
#my moudles
from DRP_utils import data_preprocessing as dp_nb
reload(dp_nb)
import Learning_curve_xgboost as lc_nb
reload(lc_nb)

from DRP_utils import testing as t_nb
reload(t_nb)
import Data_imports as di_nb
reload(di_nb)
import unseen_train_test_split as tts_nb
reload(tts_nb)

<module 'unseen_train_test_split' from '/data/home/wpw035/Drug_response_prediction/DRP-alpha-phos-rna-prot-intersect/XGboost/Unseen_cell_line_testing/unseen_train_test_split.py'>

In [3]:
#input data
prot, rna, phospho_ls, one_hot_cls, one_hot_drugs, ic50_df1 = di_nb.read_input_data()
_all_cls = prot.index
_all_drugs = ic50_df1.columns
assert prot.shape[0] == rna.shape[0] == phospho_ls.shape[0]
assert phospho_ls.shape[0]  == one_hot_cls.shape[0]
prot.shape, rna.shape, phospho_ls.shape, one_hot_cls.shape, one_hot_drugs.shape

/data/home/wpw035/.conda/envs/xgboostenv/lib/python3.7/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


Number of missing prot values 0.386335609896865
num non overlapping prot and target cls: 10
num non overlapping cls: 930


((38, 8457), (38, 17417), (38, 22804), (38, 38), (345, 345))

## Featrue selection (FS) and data createing for each drug

In [4]:
#read in landmark genes for fs and find landmarks that overlap with rna data
landmark_genes = pd.read_csv(
    f'{codebase_path}/downloaded_data_small/landmark_genes_LINCS.txt',sep='\t')
landmark_genes.index = landmark_genes['Symbol']

overlapping_landmarks, _ = dp_nb.keep_overlapping(
    pd.DataFrame(landmark_genes['Symbol']), rna.T)

overlapping_landmarks = overlapping_landmarks['Symbol'].values

#create input data for each drug
x_all, x_drug, y_list = dp_nb.create_all_drugs(
    rna[overlapping_landmarks], one_hot_drugs, ic50_df1, _all_cls)

x_all = x_all.astype(np.float32)
x_drug = x_drug.astype(np.float16)

#fmt index to include drug cell line paris
cls_drugs_index = x_all.index + '::' + x_drug.index
x_all.index = cls_drugs_index
x_drug.index = cls_drugs_index
y_list.index = cls_drugs_index

x_all.shape, x_drug.shape, len(y_list)

((11583, 908), (11583, 345), 11583)

In [6]:
#use the same landmark genes, that were used for fs for rna datan
#for fs with prot data

#find overlapping landmark genes and prot features
overlapping_landmarks, _ = dp_nb.keep_overlapping(
    pd.DataFrame(landmark_genes['Symbol']), prot.T)

overlapping_landmarks = overlapping_landmarks['Symbol'].values

#create prot data for all drugs
x_all_prot, x_drug, y_list = dp_nb.create_all_drugs(
    prot[overlapping_landmarks], one_hot_drugs, ic50_df1, _all_cls)

#fmt index to include drug cell line paris
cls_drugs_index = x_all_prot.index + '::' + x_drug.index 
x_all_prot.index = cls_drugs_index
y_list.index = cls_drugs_index
x_drug.index = cls_drugs_index

x_all_prot = x_all_prot.astype(np.float32)
x_all_prot.shape

(11583, 721)

In [7]:
#one hot data creation for all drugs
x_hot, x_drug_hot, y_hot = dp_nb.create_all_drugs(
    one_hot_cls, one_hot_drugs, ic50_df1, _all_cls)

cls_drugs_index_hot = x_hot.index + '::' + x_drug_hot.index 

x_hot.index = cls_drugs_index_hot
x_hot.columns = np.arange(len(x_drug.columns), len(x_drug.columns) + len(x_hot.columns))
x_hot.shape

(11583, 38)

## TO do add phospo FS

# Learning curves 

## set hyper parm and train size search space

In [8]:

params = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'learning_rate': [0.1, 0.2, 0.3],
           'subsample':[0.7, 0.8, 0.9, 1.0],
           'colsample_bytree':[ 0.4, 0.6, 0.8, 1.0],
           'n_estimators': [25000],
           'reg_alpha': [0, 0.1, 1, 10],
            'reg_lambda': [0, 0.1, 1, 10]
         }



parm_grid = sklearn.model_selection.ParameterGrid(params)

In [9]:
_train_size  = 0.6 #train size relative to total data set size
lg_space = np.logspace(1, np.log2(64), base=2.0, num=5).astype(int)
lg2 = np.logspace(np.log2(64), np.log2(len(x_all) * _train_size),  base=2.0, num=50).astype(int)
lg_space = np.concatenate((lg_space, lg2))
lg_space = np.unique(lg_space)
lg_space

array([   2,    4,   11,   26,   64,   70,   77,   85,   93,  103,  113,
        125,  137,  151,  166,  183,  201,  221,  244,  268,  295,  325,
        358,  394,  433,  477,  525,  577,  635,  699,  769,  847,  932,
       1025, 1128, 1242, 1366, 1503, 1654, 1821, 2003, 2205, 2426, 2669,
       2937, 3232, 3557, 3914, 4307, 4740, 5215, 5739, 6315, 6949])

In [16]:
49 + 31

80

In [ ]:
N = 31

t1 = time.time()
for run in range(N):
    run = run + 49
    print(f'run {run} of {N}')
    #test train split
    rand_seed = 42 + run
    pairs_with_truth_vals =  y_list.index
    train_pairs, test_pairs, val_pairs = tts_nb.split(
        rand_seed, _all_cls, _all_drugs, pairs_with_truth_vals, 
        train_size=_train_size )

    #rna test train selection
    x_train_rna, x_test_rna = x_all.loc[train_pairs], x_all.loc[test_pairs]
    x_val_rna = x_all.loc[val_pairs]
    y_train, y_test = y_list[train_pairs], y_list[test_pairs]
    y_val = y_list[val_pairs]
    xdrug_train, xdrug_test = x_drug.loc[train_pairs], x_drug.loc[test_pairs]
    xdrug_val = x_drug.loc[val_pairs]

    #prot test train selection
    x_train_prot, x_test_prot = x_all_prot.loc[train_pairs], x_all_prot.loc[test_pairs]
    x_val_prot = x_all_prot.loc[val_pairs]

    #one hot test train seleciton
    x_train_hot, x_test_hot = x_hot.loc[train_pairs], x_hot.loc[test_pairs]
    x_val_hot = x_hot.loc[val_pairs]
    
    #concat of omic and drug data
    x_train_rna = pd.concat((x_train_rna, xdrug_train), axis=1)
    x_val_rna = pd.concat((x_val_rna, xdrug_val), axis=1)
    x_test_rna = pd.concat((x_test_rna, xdrug_test), axis=1)
    
    x_train_prot = pd.concat((x_train_prot, xdrug_train), axis=1)
    x_val_prot = pd.concat((x_val_prot, xdrug_val), axis=1)
    x_test_prot = pd.concat((x_test_prot, xdrug_test), axis=1)
    
    x_train_hot = pd.concat((x_train_hot, xdrug_train), axis=1)
    x_val_hot = pd.concat((x_val_hot, xdrug_val), axis=1)
    x_test_hot = pd.concat((x_test_hot, xdrug_test), axis=1)

    #consistencey checks
    assert (x_train_hot.index == x_train_rna.index).all()
    assert (x_test_hot.index == x_test_rna.index).all()
    assert (x_val_hot.index == x_val_rna.index).all()

    assert (x_train_prot.index == x_train_rna.index).all()
    assert (x_test_prot.index == x_test_rna.index).all()
    assert (x_val_prot.index == x_val_rna.index).all()

    assert (y_train.index == x_train_rna.index).all()
    assert (y_test.index == x_test_rna.index).all()
    assert (xdrug_test.index == x_test_rna.index).all()

    #inconsistencey checks
    assert x_train_rna.shape[1] != x_train_prot.shape[1]
    assert x_test_rna.shape[1] != x_test_prot.shape[1]
    assert x_val_rna.shape[1] != x_val_prot.shape[1]

    assert x_train_rna.shape[1] != x_train_hot.shape[1]
    assert x_test_rna.shape[1] != x_test_hot.shape[1]
    assert x_val_rna.shape[1] != x_val_hot.shape[1]

    assert x_train_prot.shape[1] != x_train_hot.shape[1]
    assert x_test_prot.shape[1] != x_test_hot.shape[1]
    
    del x_train_hot, x_val_hot, x_test_hot
    del x_train_prot, x_val_prot, x_test_prot
    
    data_type = 'RNA'
    model_path = f'optimal-models/{data_type}/run{run}model_train_size_'
    mse_r2, bms, bhps = lc_nb.run_lc_xg_ucl(
        model_func=xgb.XGBRegressor,
        param_grid = parm_grid,
        xtrain=x_train_rna, 
        ytrain=y_train, 
        xval=x_val_rna, 
        yval=y_val, 
        xtest=x_test_rna,
        ytest=y_test, 
        train_sizes=lg_space, 
        num_trails=15,
        es_rounds=500,
        model_save_path=model_path
)
    
    #save data
    #--------- Dont overwrite exiting data ------------    
    mse_r2.to_csv(f'LC-metric-results/{data_type}/run{run}')
    d = {}
    for hp in bhps:
        for k, v in hp.items():
            if k in d.keys():
                d[k].append(v)
            else:
                d[k] = [v]
    bhps_df = pd.DataFrame(d)
    bhps_df.to_csv(f'Optimal-hyperparameters/{data_type}/run{run}df')
    with open(f'Optimal-hyperparameters/{data_type}/run{run}.pkl', 'wb') as f:
        pickle.dump(bhps, f)
    np.savetxt(f'test_train_cls/{data_type}/train_pairs{run}', train_pairs, fmt='%s')
    np.savetxt(f'test_train_cls/{data_type}/test_pairs{run}', test_pairs, fmt='%s')
    np.savetxt(f'test_train_cls/{data_type}/val_pairs{run}', val_pairs, fmt='%s')
    
    del bms, bhps
    
    

run 49 of 31
Fraction of cls in sets, relative to all clsbefore mising values are removed
train fraction 0.5789473684210527, test fraction 0.21052631578947367,validaiton fraciton 0.21052631578947367
------
Fraction of cls in sets, relative to all cl drug pairs, aftermising values are removed
train fraction 0.5093821510297483, test fraction0.19138062547673532, validaiton fraciton 0.18276125095347062
1 of 54 train sizes


/data/home/wpw035/.conda/envs/xgboostenv/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


2 of 54 train sizes
3 of 54 train sizes
4 of 54 train sizes
5 of 54 train sizes
6 of 54 train sizes
7 of 54 train sizes
8 of 54 train sizes
9 of 54 train sizes
10 of 54 train sizes
11 of 54 train sizes
12 of 54 train sizes
13 of 54 train sizes
14 of 54 train sizes
15 of 54 train sizes
16 of 54 train sizes
17 of 54 train sizes
18 of 54 train sizes
19 of 54 train sizes
20 of 54 train sizes
21 of 54 train sizes
22 of 54 train sizes
23 of 54 train sizes
24 of 54 train sizes
25 of 54 train sizes
26 of 54 train sizes
27 of 54 train sizes
28 of 54 train sizes
29 of 54 train sizes
30 of 54 train sizes
31 of 54 train sizes
32 of 54 train sizes
33 of 54 train sizes
34 of 54 train sizes
35 of 54 train sizes
36 of 54 train sizes
37 of 54 train sizes
38 of 54 train sizes
39 of 54 train sizes
40 of 54 train sizes
41 of 54 train sizes
42 of 54 train sizes
43 of 54 train sizes
44 of 54 train sizes
45 of 54 train sizes
46 of 54 train sizes
47 of 54 train sizes
48 of 54 train sizes
49 of 54 train sizes


/data/home/wpw035/.conda/envs/xgboostenv/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


2 of 54 train sizes
3 of 54 train sizes
4 of 54 train sizes
5 of 54 train sizes
6 of 54 train sizes
7 of 54 train sizes
8 of 54 train sizes
9 of 54 train sizes
10 of 54 train sizes
11 of 54 train sizes
12 of 54 train sizes
13 of 54 train sizes
14 of 54 train sizes
15 of 54 train sizes
16 of 54 train sizes
17 of 54 train sizes
18 of 54 train sizes
19 of 54 train sizes
20 of 54 train sizes
21 of 54 train sizes
22 of 54 train sizes
23 of 54 train sizes
24 of 54 train sizes
25 of 54 train sizes
26 of 54 train sizes
27 of 54 train sizes
28 of 54 train sizes
29 of 54 train sizes
30 of 54 train sizes
31 of 54 train sizes
32 of 54 train sizes
33 of 54 train sizes
34 of 54 train sizes
35 of 54 train sizes
36 of 54 train sizes
37 of 54 train sizes
38 of 54 train sizes
39 of 54 train sizes
40 of 54 train sizes
41 of 54 train sizes
42 of 54 train sizes
43 of 54 train sizes
44 of 54 train sizes
45 of 54 train sizes
46 of 54 train sizes
47 of 54 train sizes
48 of 54 train sizes
49 of 54 train sizes


...

2 of 54 train sizes
3 of 54 train sizes
4 of 54 train sizes
5 of 54 train sizes
6 of 54 train sizes
7 of 54 train sizes
8 of 54 train sizes
9 of 54 train sizes
10 of 54 train sizes
11 of 54 train sizes
12 of 54 train sizes
13 of 54 train sizes
14 of 54 train sizes
15 of 54 train sizes
16 of 54 train sizes
17 of 54 train sizes
18 of 54 train sizes
19 of 54 train sizes
20 of 54 train sizes
21 of 54 train sizes
22 of 54 train sizes
23 of 54 train sizes
24 of 54 train sizes
25 of 54 train sizes
26 of 54 train sizes
27 of 54 train sizes
28 of 54 train sizes
29 of 54 train sizes
30 of 54 train sizes
31 of 54 train sizes
32 of 54 train sizes
33 of 54 train sizes
34 of 54 train sizes
35 of 54 train sizes
36 of 54 train sizes
37 of 54 train sizes
38 of 54 train sizes
39 of 54 train sizes
40 of 54 train sizes
41 of 54 train sizes
42 of 54 train sizes
43 of 54 train sizes
44 of 54 train sizes
45 of 54 train sizes
46 of 54 train sizes
47 of 54 train sizes
48 of 54 train sizes
49 of 54 train sizes


/data/home/wpw035/.conda/envs/xgboostenv/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


2 of 54 train sizes
3 of 54 train sizes
4 of 54 train sizes
5 of 54 train sizes
6 of 54 train sizes
7 of 54 train sizes
8 of 54 train sizes
9 of 54 train sizes
10 of 54 train sizes
11 of 54 train sizes
12 of 54 train sizes
13 of 54 train sizes
14 of 54 train sizes
15 of 54 train sizes
16 of 54 train sizes
17 of 54 train sizes
18 of 54 train sizes
19 of 54 train sizes
20 of 54 train sizes
21 of 54 train sizes
22 of 54 train sizes
23 of 54 train sizes
24 of 54 train sizes
25 of 54 train sizes
26 of 54 train sizes
27 of 54 train sizes
28 of 54 train sizes
29 of 54 train sizes
30 of 54 train sizes
31 of 54 train sizes
32 of 54 train sizes
33 of 54 train sizes
34 of 54 train sizes
35 of 54 train sizes
36 of 54 train sizes
37 of 54 train sizes
38 of 54 train sizes
39 of 54 train sizes
40 of 54 train sizes
41 of 54 train sizes
42 of 54 train sizes
43 of 54 train sizes
44 of 54 train sizes
45 of 54 train sizes
46 of 54 train sizes
47 of 54 train sizes
48 of 54 train sizes
49 of 54 train sizes


/data/home/wpw035/.conda/envs/xgboostenv/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


2 of 54 train sizes
3 of 54 train sizes
4 of 54 train sizes
5 of 54 train sizes
6 of 54 train sizes
7 of 54 train sizes
8 of 54 train sizes
9 of 54 train sizes
10 of 54 train sizes
11 of 54 train sizes
12 of 54 train sizes
13 of 54 train sizes
14 of 54 train sizes
15 of 54 train sizes
16 of 54 train sizes
17 of 54 train sizes
18 of 54 train sizes
19 of 54 train sizes
20 of 54 train sizes
21 of 54 train sizes
22 of 54 train sizes
23 of 54 train sizes
24 of 54 train sizes
25 of 54 train sizes
26 of 54 train sizes
27 of 54 train sizes
28 of 54 train sizes
29 of 54 train sizes
30 of 54 train sizes
31 of 54 train sizes
32 of 54 train sizes
33 of 54 train sizes
34 of 54 train sizes
35 of 54 train sizes
